In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.utils import *
from scipy import *
from keras.optimizers import *

import xlrd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def str2int(str):
    for i,s in enumerate(str):
        if s == ' ':
            return int(str[0:i])

def img_trans(file_name):
    width = 256
    height = 256
    channel = 1
    image = Image.open(file_name)
    image = np.array(image.convert("L"))

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image[i, j] = np.int_(np.power((max(image[i, j], 170)-170)*3/255,0.4)*255)
    pil_im = Image.fromarray(image)
    out = pil_im.resize((width,height))
    data = out.getdata()
    data = np.matrix(data, dtype='float')/256
    new_data = np.reshape(data, (width,height,1))
    return new_data

def get_inform():

    longitude = []
    latitude = []
    img_list = []
    img_longth = []
    path = "dataset/dataset/infos/north/"
    path_img = "dataset/dataset/imgs/north/"

    file_list = os.listdir(path)
    for file in file_list:
        xlsfile = path + file
        book = xlrd.open_workbook(xlsfile)
        sheet0 = book.sheet_by_index(0)
        row4_data = sheet0.col_values(2)[:]
        row5_data = sheet0.col_values(3)[:]
        img_longth.append(len(row4_data)-1)

        for i in range(1,len(row4_data)):
            p = float(row4_data[i])
            w = float(row5_data[i])

            longitude.append(p)
            latitude.append(w)


    file_list = os.listdir(path_img)
    img_l = 0
    for file in file_list:
        print(file)
        xlsfile = path_img + file + "/"
        for i in range(img_longth[img_l]):
            img_list.append(img_trans(xlsfile + str(i+1)+ "_size256.jpg"))
        img_l = img_l + 1

    path = "dataset/dataset/infos/south/"
    path_img = "dataset/dataset/imgs/south/"
    img_longth = []
    file_list = os.listdir(path)
    for file in file_list:
        xlsfile = path + file
        book = xlrd.open_workbook(xlsfile)
        sheet0 = book.sheet_by_index(0)
        row4_data = sheet0.col_values(2)[:]
        row5_data = sheet0.col_values(3)[:]
        img_longth.append(len(row4_data) - 1)

        for i in range(1, len(row4_data)):
            p = float(row4_data[i])
            w = float(row5_data[i])

            longitude.append(p)
            latitude.append(w)
    print(len(longitude))

    position = np.zeros((len(longitude), 2))
    for i in range(len(longitude)):
        position[i, 0] = longitude[i] + 180
        position[i, 1] = latitude[i]


    print(position)

    file_list = os.listdir(path_img)
    img_l = 0
    for file in file_list:
        print(file)
        xlsfile = path_img + file + "/"
        for i in range(img_longth[img_l]):
            img_list.append(img_trans(xlsfile + str(i + 1) + "_size256.jpg"))
        img_l = img_l + 1

    img_list = np.array(img_list)
    lens = len(img_list)
    img_list.shape = (lens, 256, 256, 1)
    lens = int(np.ceil(lens * 0.9))
    train_img = img_list[:lens]
    test_img = img_list[lens:]
    train_pos = position[:lens]
    test_pos = position[lens:]

    print(np.array(img_list).shape)
    print(position.shape)


    return train_img,test_img,train_pos,test_pos


distance = 12756

def model():
    col = 256
    row = 256
    channel = 1
    m = Sequential()
    m.add(Conv2D(64, kernel_size=(5, 5), strides=2, padding='same', input_shape=(256, 256, 1), activation='relu'))
    m.add(MaxPool2D())
    m.add(Conv2D(32, kernel_size=(5, 5), strides=2, padding='same', activation='relu'))
    m.add(MaxPool2D())
    m.add(Conv2D(16, kernel_size=(2, 2), padding='same', strides=1, activation='relu'))
    m.add(MaxPool2D())

    m.add(Conv2D(1, kernel_size=(2, 2), padding='same', strides=1, activation='relu'))
    m.add(MaxPool2D())

    m.add(Flatten(input_shape=m.output_shape[1:]))

    m.add(Dense(64, activation='sigmoid'))
    m.add(BatchNormalization())

    m.add(Dense(16, activation='sigmoid'))
    m.add(BatchNormalization())
    m.add(Dense(2))
    m.summary()

    return m


def train(x,y):

    m=model()
    m.compile(optimizer=rmsprop(lr=0.1),loss = 'mse',metrics=['accuracy'])
    m.fit(x,y,epochs=1)
    return m

def test(y,y_):
    y_d = (y-y_)*(y-y_)

    print(y)
    print(y_)
    return np.average(y_d)



Using TensorFlow backend.


In [2]:
train_img,test_img,train_pos,test_pos = get_inform()

200101


200102


200103


200104


200105


...

200202


200203


200204


200205


200206


In [ ]:
print(train_pos)
m = train(train_img,train_pos)
y = m.predict(test_img)
print(test(y,test_pos))


200101


200102


200103


200104


200105


...

200202


200203


200204


200205


200206
